In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [2]:
data=pd.read_csv('kc_house_data.csv')
train=pd.read_csv('wk3_kc_house_train_data.csv')
test=pd.read_csv('wk3_kc_house_test_data.csv')

In [11]:
data.shape

(21613, 21)

In [12]:
def get_numpy_data(df, feat, out):
    df['constant'] = 1                          # add a constant col
    feat = ['constant'] + feat
    
    feat_mat=df.loc[:,feat]
    feat_mat = np.array(feat_mat)
    
    out_arr=df.loc[:,out]
    out_arr = np.array(out_arr).reshape(-1,)
    
    return feat_mat, out_arr

In [13]:
def predict_outcome(feat_mat, wts):
    return np.dot(feat_mat,wts)

In [14]:
def normalize_features(feat_mat):
    
    norms = np.linalg.norm(feat_mat, axis=0)
    feat_mat = feat_mat/norms
    
    return feat_mat, norms

## effect of l1 penalty

In [15]:
feat = ['sqft_living','bedrooms']
out = ['price']
feat_mat ,out_arr = get_numpy_data(train,feat,out)

In [19]:
norm_feat , norms = normalize_features(feat_mat)

In [20]:
norm_feat.shape

(9761, 3)

In [21]:
norms

array([9.87977733e+01, 2.25597973e+05, 3.46770818e+02])

In [22]:
init_wts=[1.,4.,1.]
pred=predict_outcome(norm_feat,init_wts)

In [23]:
pred=pred.reshape(-1,1)
print(pred.shape)

(9761, 1)


In [24]:
out_arr=out_arr.reshape(-1,1)

In [25]:
# calculating ro for coordinate descent
m,n=norm_feat.shape
print(m,n)
ro=np.zeros((n,))
print(ro.shape)
for i in range(n):
    ro[i] = np.sum( norm_feat[:,i]*(out_arr - pred + init_wts[i]*norm_feat[:,i] ) )

9761 3
(3,)


In [26]:
ro

array([5.23394582e+11, 4.78308761e+11, 5.03409379e+11])

In [27]:
## if l1_pen in between -1.56/2 and 1.56/2, w[2]=0 but w[1]!=0
## all range > 1.56/2 set both to 0

## Coordinate descent

In [28]:
## single step of coordinate descent

def lasso_coor_desc_step(i, feat_mat, out, wts, l1_pen):
    
    pred = predict_outcome(feat_mat,wts)      # compute prediction
    
    norm_feat , norms = normalize_features(feat_mat)
    
    ro_i = np.sum( norm_feat[:,i]*(out - pred + wts[i]*norm_feat[:,i] ) )     # compute ro[i]
    
    if i == 0:
        new_wt = ro_i
    elif ro_i < -l1_pen/2.:
        new_wt = ro_i + l1_pen/2.
    elif ro_i > l1_pen/2.:
        new_wt = ro_i - l1_pen/2.
    else:
        new_wt = 0.0
    
    return new_wt

In [29]:
# should print 0.425558846691
import math
lasso_coor_desc_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910257

## Cyclic coordinate descent

In [34]:
def lasso_cyc_coor_desc(feat_mat, out, init_wts, l1_pen, tol):
    m,n=feat_mat.shape
    wts=np.array(init_wts)
    converged=False
    
    while not converged:
        y_pred=predict_outcome(feat_mat,wts)
        error= np.sum( (out-y_pred)**2 )
        diff=0.0
        max_change=0.0
        
        for j in range(n):
            prev=wts[j]
            wts[j]=lasso_coor_desc_step(j,feat_mat,out,wts,l1_pen)
            diff= np.abs(prev-wts[j])
            max_change=max(max_change,diff)
            
        if max_change<tol:
            converged=True
    return wts

In [35]:
feat = ['sqft_living','bedrooms']
out = ['price']
feat_mat ,out_arr = get_numpy_data(train,feat,out)
norm_feat , norms = normalize_features(feat_mat)

In [36]:
init_wts=[0.0,0.0,0.0]
l1_pen=1e7
tol=1.0

In [38]:
wts=lasso_cyc_coor_desc(feat_mat,out_arr,init_wts,l1_pen,tol)

In [ ]:
wts

In [ ]:
pred=predict_outcome(feat_mat,wts)

In [ ]:
## RSS
e= np.sum( (out_arr-pred)**2 )

## more features

In [ ]:
feat = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
out = ['price']
feat_mat ,out_arr = get_numpy_data(train,feat,out)
norm_feat , norms = normalize_features(feat_mat)

In [ ]:
init_wts=[0.0,0.0,0.0]
l1_pen=1e7
tol=1.0

In [ ]:
wts1=lasso_cyc_coor_desc(feat_mat,out_arr,init_wts,l1_pen,tol)
wts1

In [ ]:
l1_pen=1e8
wts2=lasso_cyc_coor_desc(feat_mat,out_arr,init_wts,l1_pen,tol)
wts2

In [ ]:
l1_pen=1e4
wts3=lasso_cyc_coor_desc(feat_mat,out_arr,init_wts,l1_pen,tol)
wts3

In [40]:
## weights for original features so that we dont have to normalise test set
wts1_n=wts1/norms
wts2_n=wts1/norms
wts3_n=wts1/norms

In [ ]:
## test on test data
feat_mat ,out_arr = get_numpy_data(test,feat,out)

In [ ]:
pred1=predict_outcome(feat_mat,wts1_n)
pred2=predict_outcome(feat_mat,wts2_n)
pred3=predict_outcome(feat_mat,wts3_n)

In [ ]:
## RSS
e1= np.sum( (out_arr-pred1)**2 )
e2= np.sum( (out_arr-pred2)**2 )
e3= np.sum( (out_arr-pred3)**2 )

In [ ]:
e1

In [ ]:
e2

In [ ]:
e3